In [48]:
import os
import sys
import random
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import sys
import io
from contextlib import redirect_stdout
import matplotlib.pyplot as plt
import pandas as pd

# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

Using device: cuda


## Prepare the dataset

In [72]:
# Example DataFrame loading
df = pd.read_csv('edinburgh_trajectories.csv')
num_cols = df.shape[1]
position_indices = [i for i in range(num_cols) if i % 4 == 1 or i % 4 == 2]
position_df = df.iloc[:, position_indices]
position_array = position_df.to_numpy()
sequence_length = len(position_indices) // 2
tensor_list = []

for row in position_array:
    reshaped_tensor = torch.tensor(row.reshape(sequence_length, 2))
    tensor_list.append(reshaped_tensor)

all_trajectories_tensor = torch.stack(tensor_list) / 10

def missing(x, y, z, p):
    return torch.tensor(np.repeat(np.random.rand(x * y) > p, z).reshape(x, y, z)).float()

def generate_masks(tensors, min_mask_ratio=0.2, max_mask_ratio=0.4, missing_ratio=0.5):
    initial_masks = missing(tensors.shape[0], tensors.shape[1], tensors.shape[2], missing_ratio)
    masks = []
    for initial_mask in initial_masks:
        seq_length = initial_mask.shape[0]
        mask_start = np.random.randint(int(seq_length * min_mask_ratio), int(seq_length * max_mask_ratio))
        mask = torch.zeros_like(initial_mask)
        mask[:, :mask_start] = 1
        mask = initial_mask * mask
        mask[0] = 1
        mask[1] = 1
        masks.append(mask.tolist())
    return torch.tensor(masks)

# split the data into training and validation sets
train_ratio = 0.8
train_size = int(train_ratio * all_trajectories_tensor.shape[0])
train_trajectories_tensor = all_trajectories_tensor[:train_size]
val_trajectories_tensor = all_trajectories_tensor[train_size:]

input_mask = generate_masks(val_trajectories_tensor)
outputs_mask = generate_masks(val_trajectories_tensor)

class trainDataset(Dataset):
    def __init__(self, tensor):
        self.tensor = tensor.float().to(device)
    def __len__(self):
        return len(self.tensor)
    def __getitem__(self, idx):
        return self.tensor[idx]

class valDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs.float().to(device)
        self.outputs = outputs.float().to(device)
    def __len__(self):
        return len(self.inputs)
    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

train_dataset = trainDataset(train_trajectories_tensor)
val_inputs = val_trajectories_tensor * input_mask
val_outputs = val_trajectories_tensor * outputs_mask
val_dataset = valDataset(val_inputs, val_outputs)

dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [78]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x, h0=None):
        if h0 is None:
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, hidden = self.gru(x, h0)  
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        return out, hidden

class CustomMSE(nn.Module):
    def __init__(self):
        super(CustomMSE, self).__init__()

    def forward(self, output, target):
        # where target is 0
        mask = (target != 0).float().to(device)
        return torch.mean(((output - target) ** 2) * mask)

class CustomMAE(nn.Module):
    def __init__(self):
        super(CustomMAE, self).__init__()

    def forward(self, output, target):
        # where target is 0
        mask = (target != 0).float().to(device)
        return torch.mean(torch.abs(output - target) * mask)

In [87]:
def train_model(model, dataloader, epochs, optimizer, criterion):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs in dataloader:
            optimizer.zero_grad()
            hidden = None
            loss = 0

            # Autoregressive prediction
            # Start with the first input and predict each subsequent step
            seq_len = inputs.size(1)
            current_input = inputs[:, 0, :].unsqueeze(1) # start with the first input step
            for t in range(seq_len):
                prediction, hidden = model(current_input, hidden)
                previous_input = current_input.squeeze(1)
                current_input = inputs[:, t, :].unsqueeze(1)  # next input comes from the ground truth
                loss += criterion(prediction - previous_input, inputs[:, t, :]-previous_input) #(current_input-previous_input).squeeze(1))

            loss.backward()
            optimizer.step()
            total_loss += loss.item()
 
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

model = GRUModel(input_size=2, hidden_size=128, num_layers=2, num_classes=2).to(device)
dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CustomMSE()
train_model(model, dataloader, 20, optimizer, criterion)

Epoch 1, Loss: 1374853.30625
Epoch 2, Loss: 1025936.3
Epoch 3, Loss: 572232.4328125
Epoch 4, Loss: 341278.2828125
Epoch 5, Loss: 298642.97109375
Epoch 6, Loss: 291074.603125
Epoch 7, Loss: 274704.97109375
Epoch 8, Loss: 161413.95703125
Epoch 9, Loss: 93920.583203125
Epoch 10, Loss: 66604.6375
Epoch 11, Loss: 43209.09755859375
Epoch 12, Loss: 24345.62099609375
Epoch 13, Loss: 14646.72861328125
Epoch 14, Loss: 10175.724462890625
Epoch 15, Loss: 8986.30107421875
Epoch 16, Loss: 8526.274243164062
Epoch 17, Loss: 8314.109497070312
Epoch 18, Loss: 8250.988818359376
Epoch 19, Loss: 8037.2176513671875
Epoch 20, Loss: 8190.695727539062


In [91]:
def checkModel(model, inputs):
    model.eval()
    with torch.no_grad():
        hidden = None
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(seq_len):
            prediction, hidden = model(current_input, hidden)
            if t > 5:
                current_input = prediction.unsqueeze(1)
            else:
                current_input = inputs[:, t, :].unsqueeze(1)
            print(prediction[2], inputs[:, t, :][2])

# get the first batch in dataloader
inputs = next(iter(dataloader))
checkModel(model, inputs)

tensor([125.6118,  37.2211], device='cuda:0') tensor([102.4169,  58.9456], device='cuda:0')
tensor([103.6906,  60.2194], device='cuda:0') tensor([102.5772,  59.0529], device='cuda:0')
tensor([98.0040, 58.3232], device='cuda:0') tensor([102.9224,  59.2840], device='cuda:0')
tensor([102.6283,  56.1933], device='cuda:0') tensor([103.4246,  59.6078], device='cuda:0')
tensor([104.6597,  58.3398], device='cuda:0') tensor([104.1253,  60.0554], device='cuda:0')
tensor([103.7296,  60.0960], device='cuda:0') tensor([104.8880,  60.5386], device='cuda:0')
tensor([103.8274,  60.6707], device='cuda:0') tensor([105.6763,  61.0188], device='cuda:0')
tensor([102.9108,  61.0683], device='cuda:0') tensor([106.4125,  61.4672], device='cuda:0')
tensor([102.0657,  61.4929], device='cuda:0') tensor([107.1865,  61.8727], device='cuda:0')
tensor([101.3570,  61.9344], device='cuda:0') tensor([108.0303,  62.2991], device='cuda:0')
tensor([100.7646,  62.3816], device='cuda:0') tensor([108.8609,  62.6971], device=

In [ ]:
def eval(model, dataloader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs in dataloader:
            hidden = None
            loss = 0
            seq_len = inputs.size(1)
            current_input = inputs[:, 0, :].unsqueeze(1)

            for t in range(seq_len):
                prediction, hidden = model(current_input, hidden)
                current_input = inputs[:, t, :].unsqueeze(1)
                loss += criterion(prediction, inputs[:, t, :])

            total_loss += loss.item()

    print(f'Validation Loss: {total_loss / len(dataloader)}')